In [1]:
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
utils = importr('utils')


In [3]:
utils.install_packages('pscl')
utils.install_packages('MASS')
utils.install_packages('stats')
#utils.install_packages('xlsx')
utils.install_packages('lmtest')
utils.install_packages('nonnest2')

RRuntimeError: Error in contrib.url(repos, "source") : 
  trying to use CRAN without setting a mirror


In [2]:
MASS = importr('MASS')
stats = importr('stats')
#xlsx = importr('xlsx')
pscl = importr('pscl')
base = importr('base')
lmtest = importr('lmtest')
nonnest2 = importr('nonnest2')

In [17]:
import pandas as pd

table = pd.read_excel('C:\\Users\\utente\\PanelJamDataExtractor\\TabellaCompleta.xlsx', index = False)
table = table.drop(columns = ['autore_panel_attuale'])
time = table['time'].tolist()
numTime = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))

featureTable = table
featureTable['time'] = numTime
logit = featureTable.drop(columns = ['time','project_depth','stars_attuale','followers_attuale','tot_loves_attuale','tot_views_attuale',
                'ranking_attuale','Likes','Views','Comments'])
numerical = featureTable.drop(columns = ['id_prog','id_panel','remixed','has_avatar_attuale','has_bio_attuale'])



In [18]:
numerical['project_depth'] = numerical['project_depth'] - 1

In [19]:
numerical

,time,project_depth,stars_attuale,followers_attuale,tot_loves_attuale,tot_views_attuale,ranking_attuale,Likes,Views,Comments
0,75,0,221,4,179,129504,1998.584615,4,938,3
1,75,1,6943,31,1554,797095,1096.085714,4,938,3
2,75,2,9521,15,826,454623,1233.381963,4,938,3
3,90,0,221,4,179,129504,1998.584615,1,762,0
4,90,1,6943,31,1554,797095,1096.085714,1,762,0
...,...,...,...,...,...,...,...,...,...,...
14835,880,1,1,0,2,155,79.000000,1,68,1
14836,880,0,0,0,1,110,111.000000,1,68,1
14837,1060,2,4,2,21,620,107.833333,3,67,0
14838,1060,1,0,0,10,287,99.000000,3,67,0


In [58]:
#Caso in cui si ha project_depth come variabile dipendente
import numpy as np
from sklearn.preprocessing import scale

normTable = np.log1p(numerical)
normTable['project_depth'] = numerical['project_depth'].astype(int)
#table = normTable.drop(columns = 'project_depth')
#Nessuna standardizzazione della variabile project_depth in quanto il modello di Poisson non accetta una variabile dipendente
#con valori negativi, ma solo con valori positivi ed interi.
#scalTable = pd.DataFrame(scale(table), index = table.index, columns = table.columns)
scalTable = normTable
scalTable['project_depth'] = normTable['project_depth']
#scalTable = numerical
scalTable['remixed'] = logit['remixed']
scalTable['has_avatar_attuale'] = logit['has_avatar_attuale']
scalTable['has_bio_attuale'] = logit['has_bio_attuale']


In [48]:
#Caso in cui si ha remixed come variabile dipendente
import numpy as np
from sklearn.preprocessing import scale

normTable = np.log1p(numerical)
#scalTable = pd.DataFrame(scale(normTable), index = table.index, columns = table.columns)
#I valori negativi dopo la standardizzazione sono giusti?
scalTable = normTable
scalTable['remixed'] = logit['remixed']
scalTable['has_avatar_attuale'] = logit['has_avatar_attuale']
scalTable['has_bio_attuale'] = logit['has_bio_attuale']

In [59]:
scalTable

,time,project_depth,stars_attuale,followers_attuale,tot_loves_attuale,tot_views_attuale,ranking_attuale,Likes,Views,Comments,remixed,has_avatar_attuale,has_bio_attuale
0,4.330733,0,5.402677,1.609438,5.192957,11.771475,7.600695,1.609438,6.844815,1.386294,True,True,True
1,4.330733,1,8.845633,3.465736,7.349231,13.588730,7.000413,1.609438,6.844815,1.386294,True,True,False
2,4.330733,2,9.161360,2.772589,6.717805,13.027226,7.118326,1.609438,6.844815,1.386294,False,True,True
3,4.510860,0,5.402677,1.609438,5.192957,11.771475,7.600695,0.693147,6.637258,0.000000,True,True,True
4,4.510860,1,8.845633,3.465736,7.349231,13.588730,7.000413,0.693147,6.637258,0.000000,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14835,6.781058,1,0.693147,0.000000,1.098612,5.049856,4.382027,0.693147,4.234107,0.693147,True,False,False
14836,6.781058,0,0.000000,0.000000,0.693147,4.709530,4.718499,0.693147,4.234107,0.693147,True,False,False
14837,6.966967,2,1.609438,1.098612,3.091042,6.431331,4.689818,1.386294,4.219508,0.000000,False,True,True
14838,6.966967,1,0.000000,0.000000,2.397895,5.662960,4.605170,1.386294,4.219508,0.000000,True,False,False


In [60]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()
rTable = pandas2ri.py2ri(scalTable)

In [61]:
#Caso con project_depth come variabile dipendente
formula = """project_depth ~ time + stars_attuale + remixed + followers_attuale + tot_loves_attuale +
             ranking_attuale + Likes + Views + Comments + has_bio_attuale + has_avatar_attuale"""


summary = ro.r['summary']

poissonMod = stats.glm(ro.r(formula), family = ro.r('poisson(link = "log")'), data = rTable)
nbMod = MASS.glm_nb(ro.r(formula), data = rTable)
zeroinflMod = pscl.zeroinfl(ro.r(formula), data = rTable,dist = 'negbin')
hurdleMod = pscl.hurdle(ro.r(formula), data = rTable, dist = 'negbin', zero = "binomial")
#print(dir(mod))

In [50]:
#Caso con remixed come variabile dipendente
formula2 = """remixed ~ time + project_depth + stars_attuale + followers_attuale + tot_loves_attuale +
             ranking_attuale + Likes + Views + Comments + has_bio_attuale + has_avatar_attuale"""

poissonMod2 = stats.glm(ro.r(formula2), family = ro.r('poisson()'), data = rTable)
nbMod2 = MASS.glm_nb(ro.r(formula2), data = rTable)
zeroinflMod2 = pscl.zeroinfl(ro.r(formula2), data = rTable,dist = 'negbin')
hurdleMod2 = pscl.hurdle(ro.r(formula2), data = rTable, dist = 'negbin', zero = "binomial")
#print(dir(mod))

**MODELLO DI POISSON**

In [62]:
#Modello con project_depth come variabile dipendente
poissonSum = summary(poissonMod)
loglikPoisson = stats.logLik(poissonMod)

poisson = str(poissonSum)
poisson = poisson.split('Deviance')[1]
print(poisson)
print(loglikPoisson)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-3.0509  -0.9667  -0.2424   0.5486  10.3178  



Coefficients:

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)             0.044933   0.092380   0.486 0.626691    

time                    0.055977   0.008380   6.680 2.39e-11 ***

stars_attuale          -0.015003   0.006407  -2.342 0.019205 *  

remixedTRUE            -0.816244   0.012196 -66.926  < 2e-16 ***

followers_attuale      -0.021469   0.008128  -2.642 0.008254 ** 

tot_loves_attuale       0.030275   0.007940   3.813 0.000137 ***

ranking_attuale        -0.064564   0.010202  -6.328 2.48e-10 ***

Likes                   0.065689   0.008938   7.350 1.99e-13 ***

Views                   0.158499   0.007454  21.264  < 2e-16 ***

Comments                0.072860   0.009126   7.984 1.42e-15 ***

has_bio_attualeTRUE     0.029787   0.014394   2.069 0.038505 *  

has_avatar_attualeTRUE  0.218716   0.036536   5.986 2.15e-09 ***

---

Signif. code

In [53]:
#Modello con reimxed come variabile dipendente
poissonSum2 = summary(poissonMod2)
loglikPoisson2 = stats.logLik(poissonMod2)

poisson2 = str(poissonSum2)
poisson2 = poisson2.split('Deviance')[1]
print(poisson2)
print(loglikPoisson2)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.5483  -0.1790   0.1590   0.3478   1.1037  



Coefficients:

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)            -0.102573   0.140400  -0.731   0.4650    

time                    0.006383   0.012951   0.493   0.6221    

project_depth          -0.427197   0.015651 -27.295  < 2e-16 ***

stars_attuale          -0.016674   0.010098  -1.651   0.0987 .  

followers_attuale       0.007317   0.012939   0.565   0.5717    

tot_loves_attuale       0.012448   0.012234   1.017   0.3089    

ranking_attuale        -0.045489   0.016167  -2.814   0.0049 ** 

Likes                   0.022749   0.014367   1.583   0.1133    

Views                   0.069101   0.011997   5.760 8.41e-09 ***

Comments                0.006450   0.014794   0.436   0.6628    

has_bio_attualeTRUE    -0.005747   0.022614  -0.254   0.7994    

has_avatar_attualeTRUE  0.036329   0.048787   0.745   0.4565    

---

Signif. code

**NEGATIVE BINOMIAL**

In [63]:
#Modello con project_depth come variabile dipendente
nbSum = summary(nbMod)
loglikNB = stats.logLik(nbMod)

nb = str(nbSum)
nb = nb.split('Deviance')[1]
print(nb)
print(loglikNB)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-2.7256  -0.8484  -0.2185   0.4897   7.9109  



Coefficients:

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)            -0.007202   0.103874  -0.069 0.944724    

time                    0.060060   0.009424   6.373 1.85e-10 ***

stars_attuale          -0.012885   0.007222  -1.784 0.074399 .  

remixedTRUE            -0.820694   0.014043 -58.443  < 2e-16 ***

followers_attuale      -0.022839   0.009182  -2.488 0.012864 *  

tot_loves_attuale       0.030108   0.008940   3.368 0.000758 ***

ranking_attuale        -0.066330   0.011570  -5.733 9.87e-09 ***

Likes                   0.067401   0.010118   6.662 2.70e-11 ***

Views                   0.162572   0.008468  19.199  < 2e-16 ***

Comments                0.075180   0.010325   7.281 3.31e-13 ***

has_bio_attualeTRUE     0.031863   0.016221   1.964 0.049493 *  

has_avatar_attualeTRUE  0.228669   0.040304   5.674 1.40e-08 ***

---

Signif. code

In [55]:
#Modello con remixed come variabile dipendente
nbSum2 = summary(nbMod2)
loglikNB2 = stats.logLik(nbMod2)

nb2 = str(nbSum2)
nb2 = nb2.split('Deviance')[1]
print(nb2)
print(loglikNB2)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.5483  -0.1790   0.1590   0.3478   1.1037  



Coefficients:

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)            -0.102573   0.140401  -0.731   0.4650    

time                    0.006383   0.012951   0.493   0.6221    

project_depth          -0.427197   0.015651 -27.295  < 2e-16 ***

stars_attuale          -0.016674   0.010098  -1.651   0.0987 .  

followers_attuale       0.007317   0.012939   0.565   0.5717    

tot_loves_attuale       0.012448   0.012234   1.017   0.3089    

ranking_attuale        -0.045489   0.016167  -2.814   0.0049 ** 

Likes                   0.022749   0.014367   1.583   0.1133    

Views                   0.069102   0.011997   5.760 8.41e-09 ***

Comments                0.006450   0.014794   0.436   0.6628    

has_bio_attualeTRUE    -0.005747   0.022614  -0.254   0.7994    

has_avatar_attualeTRUE  0.036329   0.048787   0.745   0.4565    

---

Signif. code

**ZERO INFLATED NEGATIVE BINOMIAL**

In [64]:
#Modello con project_depth come variabile dipendente

zeroinflSum = summary(zeroinflMod)
loglikZeroinfl = stats.logLik(zeroinflMod)

zeroinfl = str(zeroinflSum)
zeroinfl = zeroinfl.split('Pearson')[1]
print(zeroinfl)
print(loglikZeroinfl)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.9568 -0.7162 -0.2174  0.5134 18.0973 



Count model coefficients (negbin with log link):

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)             0.351154   0.107634   3.262 0.001104 ** 

time                    0.036847   0.009701   3.798 0.000146 ***

stars_attuale          -0.057169   0.007429  -7.695 1.41e-14 ***

remixedTRUE            -0.684881   0.014460 -47.363  < 2e-16 ***

followers_attuale      -0.003085   0.009722  -0.317 0.751038    

tot_loves_attuale       0.060989   0.009840   6.198 5.72e-10 ***

ranking_attuale        -0.085658   0.012046  -7.111 1.15e-12 ***

Likes                   0.059832   0.010450   5.726 1.03e-08 ***

Views                   0.191030   0.008881  21.510  < 2e-16 ***

Comments                0.079503   0.010595   7.504 6.20e-14 ***

has_bio_attualeTRUE     0.041335   0.016882   2.448 0.014348 *  

has_avatar_attualeTRUE -0.022777   0.045596  -0.500 0.61740

In [54]:
#Modello con remixed come variabile dipendente
zeroinflSum2 = summary(zeroinflMod2)
loglikZeroinfl2 = stats.logLik(zeroinflMod2)

zeroinfl2 = str(zeroinflSum2)
zeroinfl2 = zeroinfl2.split('Pearson')[1]
print(zeroinfl2)
print(loglikZeroinfl2)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.0854 -0.1528  0.1630  0.3616  8.4303 



Count model coefficients (negbin with log link):

                         Estimate Std. Error z value Pr(>|z|)    

(Intercept)            -0.0370952  0.1408790  -0.263  0.79231    

time                    0.0016154  0.0130011   0.124  0.90112    

project_depth          -0.4110676  0.0158227 -25.980  < 2e-16 ***

stars_attuale          -0.0178325  0.0101151  -1.763  0.07791 .  

followers_attuale       0.0050715  0.0129503   0.392  0.69534    

tot_loves_attuale       0.0152152  0.0122588   1.241  0.21455    

ranking_attuale        -0.0454727  0.0162172  -2.804  0.00505 ** 

Likes                   0.0230401  0.0143967   1.600  0.10951    

Views                   0.0622263  0.0120584   5.160 2.46e-07 ***

Comments                0.0007922  0.0148146   0.053  0.95735    

has_bio_attualeTRUE    -0.0026661  0.0226493  -0.118  0.90630    

has_avatar_attualeTRUE  0.0251076  0.0489449   

**HURDLE MODEL**

In [65]:
#Modello con project_depth come variabile dipendente

hurdleSum = summary(hurdleMod)
loglikHurdle = stats.logLik(hurdleMod)

hurdle = str(hurdleSum)
hurdle = hurdle.split('Pearson')[1]
print(hurdle)
print(loglikHurdle)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.8761 -0.7048 -0.2299  0.5222 18.1467 



Count model coefficients (truncated negbin with log link):

                        Estimate Std. Error z value Pr(>|z|)    

(Intercept)             0.106727   0.121669   0.877  0.38038    

time                    0.049003   0.011052   4.434 9.25e-06 ***

stars_attuale          -0.081134   0.008518  -9.525  < 2e-16 ***

remixedTRUE            -0.669121   0.016168 -41.384  < 2e-16 ***

followers_attuale       0.008435   0.010810   0.780  0.43520    

tot_loves_attuale       0.069270   0.010750   6.444 1.17e-10 ***

ranking_attuale        -0.086852   0.013560  -6.405 1.50e-10 ***

Likes                   0.074259   0.011816   6.285 3.29e-10 ***

Views                   0.218002   0.010058  21.674  < 2e-16 ***

Comments                0.102064   0.011877   8.593  < 2e-16 ***

has_bio_attualeTRUE     0.060117   0.019075   3.152  0.00162 ** 

has_avatar_attualeTRUE -0.037641   0.048734  -0.7

In [56]:
#Modello con remixed come variabile dipendente
hurdleSum2 = summary(hurdleMod2)
loglikHurdle2 = stats.logLik(hurdleMod2)

hurdle2 = str(hurdleSum2)
hurdle2 = hurdle2.split('Pearson')[1]
print(hurdle2)
print(loglikHurdle2)

 residuals:

      Min        1Q    Median        3Q       Max 

-16.66296   0.06315   0.22216   0.48654  18.05218 



Count model coefficients (truncated negbin with log link):

                         Estimate Std. Error z value Pr(>|z|)

(Intercept)              -25.7298 12275.0723  -0.002    0.998

time                       0.3915  1789.7157   0.000    1.000

project_depth             -0.4259  3950.9746   0.000    1.000

stars_attuale             -0.1268  2490.1875   0.000    1.000

followers_attuale          0.1581  3074.2572   0.000    1.000

tot_loves_attuale         -0.2858  2756.9019   0.000    1.000

ranking_attuale            0.3195  3536.3219   0.000    1.000

Likes                     -0.1131  3515.9956   0.000    1.000

Views                     -0.4148  3294.4201   0.000    1.000

Comments                  -0.1856  3751.6269   0.000    1.000

has_bio_attualeTRUE       -0.2153  4920.8648   0.000    1.000

has_avatar_attualeTRUE     1.4859 11540.0235   0.000    1.000

Lo

**LIKELIHOOD RATIO TEST E TEST DI VUONG**

**Likelihood ratio test**

In [66]:
#project_depth come variabile dipendente
lrt = lmtest.lrtest(poissonMod,nbMod)
print(lrt)

Likelihood ratio test



Model 1: project_depth ~ time + stars_attuale + remixed + followers_attuale + 

    tot_loves_attuale + ranking_attuale + Likes + Views + Comments + 

    has_bio_attuale + has_avatar_attuale

Model 2: project_depth ~ time + stars_attuale + remixed + followers_attuale + 

    tot_loves_attuale + ranking_attuale + Likes + Views + Comments + 

    has_bio_attuale + has_avatar_attuale

  #Df LogLik Df  Chisq Pr(>Chisq)    

1  12 -25420                         

2  13 -25155  1 530.98  < 2.2e-16 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



In [57]:
#remixed come variabile dipendente
lrt2 = lmtest.lrtest(poissonMod2,nbMod2)
print(lrt2)

Likelihood ratio test



Model 1: remixed ~ time + project_depth + stars_attuale + followers_attuale + 

    tot_loves_attuale + ranking_attuale + Likes + Views + Comments + 

    has_bio_attuale + has_avatar_attuale

Model 2: remixed ~ time + project_depth + stars_attuale + followers_attuale + 

    tot_loves_attuale + ranking_attuale + Likes + Views + Comments + 

    has_bio_attuale + has_avatar_attuale

  #Df LogLik Df  Chisq Pr(>Chisq)

1  12 -14011                     

2  13 -14011  1 0.1919     0.6613



**Test di Vuong**

In [104]:
#project_depth come variabile dipendente
nb_hurdle_vuong = nonnest2.vuongtest(hurdleMod,nbMod)
print(nb_hurdle_vuong)



Model 1 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.039,   p = 1.68e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 10.831,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 10.831,   p = 1



In [58]:
#remixed come variabile dipendente
nb_hurdle_vuong2 = nonnest2.vuongtest(hurdleMod2,nbMod2)
print(nb_hurdle_vuong2)



Model 1 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.392,   p = 1.2e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 107.500,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 107.500,   p = 1



In [105]:
#project_depth come variabile dipendente
zero_hurdle_test = nonnest2.vuongtest(zeroinflMod, hurdleMod)
print(zero_hurdle_test)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.013,   p = 2.13e-08



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = -5.195,   p = 1

  H1B: Model 2 fits better than Model 1 

    z = -5.195,   p = 1.023e-07



In [60]:
#remixed come variabile dipendente
zero_hurdle_test2 = nonnest2.vuongtest(zeroinflMod2, hurdleMod2)
print(zero_hurdle_test2)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.394,   p = <2e-16



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = -106.741,   p = 1

  H1B: Model 2 fits better than Model 1 

    z = -106.741,   p = < 2.2e-16



In [106]:
#project_depth come variabile dipendente
nb_zeroinfl_vuong = nonnest2.vuongtest(zeroinflMod,nbMod)
print(nb_zeroinfl_vuong)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.039,   p = 8e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 7.868,   p = 1.8e-15

  H1B: Model 2 fits better than Model 1 

    z = 7.868,   p = 1



In [66]:
#remixed come variabile dipendente
nb_zeroinfl_vuong2 = nonnest2.vuongtest(zeroinflMod2,nbMod2)
print(nb_zeroinfl_vuong2)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.003,   p = 0.00662



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 5.511,   p = 1.78e-08

  H1B: Model 2 fits better than Model 1 

    z = 5.511,   p = 1

